
# Neuron - Classificação de Emoções em Português

Este notebook implementa o pipeline completo de **Machine Learning** para o projeto Neuron:

1. Carregamento do dataset de frases em português.
2. Análise exploratória simples.
3. Pré-processamento de texto (PT-BR).
4. Criação de dois modelos de IA:
   - **Modelo 1**: classificação de **emoções** (`alegria, tristeza, raiva, medo, amor, surpresa`).
   - **Modelo 2**: classificação de **sentimento** (`positivo` x `negativo`).
5. Salvamento dos modelos em arquivo `.pkl`.
6. Exemplo de uso do modelo para rotular uma conversa ao longo de vários dias e gerar gráficos.


In [13]:
import pandas as pd
import numpy as np
import re
import nltk

from nltk.corpus import stopwords
from nltk.stem import RSLPStemmer

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, accuracy_score

import pickle
import matplotlib.pyplot as plt
import seaborn as sns

# Executar apenas na primeira vez no ambiente
# nltk.download('stopwords')
# nltk.download('rslp')


## 1. Carregar dados

In [14]:

# Ajuste o caminho se precisar
df = pd.read_csv("dados_humor_neuron_pt.csv")

df.head()


,texto,emocao
0,Acordei animado para colaborar com o time,alegria
1,Entreguei uma tarefa difícil e me senti realizado,alegria
2,Recebi elogios sinceros do time e fiquei radiante,alegria
3,O clima na daily estava leve e divertido,alegria
4,Terminei o sprint antes do prazo e comemorei,alegria


## 2. Análise exploratória simples

In [15]:
df['emocao'].value_counts()

emocao
alegria     43
medo        43
tristeza    42
surpresa    42
amor        40
raiva       39
Name: count, dtype: int64

## 3. Criar coluna de sentimento (positivo/negativo)

In [16]:

map_sent = {
    "alegria": "positivo",
    "amor": "positivo",
    "surpresa": "positivo",
    "tristeza": "negativo",
    "raiva": "negativo",
    "medo": "negativo",
}

df["sentimento"] = df["emocao"].map(map_sent)
df.head()


,texto,emocao,sentimento
0,Acordei animado para colaborar com o time,alegria,positivo
1,Entreguei uma tarefa difícil e me senti realizado,alegria,positivo
2,Recebi elogios sinceros do time e fiquei radiante,alegria,positivo
3,O clima na daily estava leve e divertido,alegria,positivo
4,Terminei o sprint antes do prazo e comemorei,alegria,positivo


## 4. Pré-processamento de texto em PT-BR

In [17]:

# garantir que os recursos do NLTK estejam disponíveis
try:
    stopwords_pt = set(stopwords.words("portuguese"))
except LookupError:
    nltk.download("stopwords")
    stopwords_pt = set(stopwords.words("portuguese"))

try:
    stemmer = RSLPStemmer()
except LookupError:
    nltk.download("rslp")
    stemmer = RSLPStemmer()

def preprocess_pt(texto: str) -> str:
    if not isinstance(texto, str):
        return ""
    texto = texto.lower()
    texto = re.sub(r"[^a-záéíóúâêôãõç\s]", " ", texto)
    tokens = texto.split()
    tokens = [t for t in tokens if t not in stopwords_pt]
    tokens = [stemmer.stem(t) for t in tokens]
    return " ".join(tokens)

df["texto_limpo"] = df["texto"].apply(preprocess_pt)
df[["texto", "texto_limpo", "emocao", "sentimento"]].head()


,texto,texto_limpo,emocao,sentimento
0,Acordei animado para colaborar com o time,acord anim colabor tim,alegria,positivo
1,Entreguei uma tarefa difícil e me senti realizado,entreg taref difícil sent realiz,alegria,positivo
2,Recebi elogios sinceros do time e fiquei radiante,receb elogi sincer tim fiq radi,alegria,positivo
3,O clima na daily estava leve e divertido,clim daily lev divert,alegria,positivo
4,Terminei o sprint antes do prazo e comemorei,termin sprint ant praz comemor,alegria,positivo


## 5. Separar treino e teste

In [18]:
train_idx, test_idx = train_test_split(
    df.index,
    test_size=0.2, random_state=42, stratify=df["emocao"]
)

X_train = df.loc[train_idx, "texto_limpo"]
X_test = df.loc[test_idx, "texto_limpo"]
y_emocao_train = df.loc[train_idx, "emocao"]
y_emocao_test = df.loc[test_idx, "emocao"]
y_sent_train = df.loc[train_idx, "sentimento"]
y_sent_test = df.loc[test_idx, "sentimento"]

len(X_train), len(X_test)


(199, 50)

## 6. Vetorização TF-IDF

In [19]:

vectorizer = TfidfVectorizer(max_features=5000)
X_train_vec = vectorizer.fit_transform(X_train)
X_test_vec  = vectorizer.transform(X_test)

X_train_vec.shape, X_test_vec.shape


((199, 383), (50, 383))

## 7. Modelo 1 – Classificação de Emoções

In [20]:

modelo_emocao = LogisticRegression(max_iter=1000, multi_class="multinomial")
modelo_emocao.fit(X_train_vec, y_emocao_train)

y_pred_emocao = modelo_emocao.predict(X_test_vec)
print("Accuracy EMOÇÃO:", accuracy_score(y_emocao_test, y_pred_emocao))
print(classification_report(y_emocao_test, y_pred_emocao))


Accuracy EMOÇÃO: 0.56
              precision    recall  f1-score   support

     alegria       0.31      0.44      0.36         9
        amor       0.60      0.75      0.67         8
        medo       0.75      0.67      0.71         9
       raiva       0.83      0.62      0.71         8
    surpresa       0.50      0.38      0.43         8
    tristeza       0.57      0.50      0.53         8

    accuracy                           0.56        50
   macro avg       0.59      0.56      0.57        50
weighted avg       0.59      0.56      0.57        50



C:\Users\Gabriel Duarte\AppData\Roaming\Python\Python313\site-packages\sklearn\linear_model\_logistic.py:1272: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.8. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(


## 8. Modelo 2 – Classificação de Sentimento (positivo/negativo)

In [21]:

modelo_sentimento = LogisticRegression(max_iter=1000)
modelo_sentimento.fit(X_train_vec, y_sent_train)

y_pred_sent = modelo_sentimento.predict(X_test_vec)
print("Accuracy SENTIMENTO:", accuracy_score(y_sent_test, y_pred_sent))
print(classification_report(y_sent_test, y_pred_sent))


Accuracy SENTIMENTO: 0.86
              precision    recall  f1-score   support

    negativo       0.91      0.80      0.85        25
    positivo       0.82      0.92      0.87        25

    accuracy                           0.86        50
   macro avg       0.87      0.86      0.86        50
weighted avg       0.87      0.86      0.86        50



## 9. Salvar modelos em arquivo `.pkl`

In [22]:

artefatos = {
    "vectorizer": vectorizer,
    "modelo_emocao": modelo_emocao,
    "modelo_sentimento": modelo_sentimento,
}

with open("modelos_neuron_pt.pkl", "wb") as f:
    pickle.dump(artefatos, f)

print("Modelos salvos em 'modelos_neuron_pt.pkl'")


Modelos salvos em 'modelos_neuron_pt.pkl'
